In [1]:
# install octave
!sudo apt-get -qq update
!sudo apt-get -qq install octave octave-signal liboctave-dev

# install oct2py that compatible with colab
import os

from pkg_resources import get_distribution

os.system(
    f"pip install -qq"
    f" ipykernel=={get_distribution('ipykernel').version}"
    f" ipython=={get_distribution('ipython').version}"
    f" tornado=={get_distribution('tornado').version}"
    f" oct2py"
)

# install packages
!pip install -qq matpower matpowercaseframes

In [2]:
import oct2py

import matpower

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.7.2
matpower version: 8.0.0.2.2.2


In [3]:
from matpower import start_instance

In [4]:
m = start_instance()

In [5]:
mpc = m.loadcase("case9")
mpopt = m.mpoption("verbose", 2)
m.push("_mpopt", mpopt)
m.push("_mpc", mpc, verbose=False)
m.eval("_r1 = runuopf(_mpc, _mpopt);", verbose=False)

In [6]:
# pull all after cleansing unsupported data
m.eval(
    """
    _r1.raw = rmfield(_r1.raw, 'task');
    _r1 = rmfield(_r1, 'om');
    """
)
mpc = m.pull("_r1")

In [7]:
# selective pull
r1_mpc = m.eval(
    "struct('baseMVA', _r1.baseMVA, 'version', _r1.version, 'bus', _r1.bus,"
    " 'gen', _r1.gen, 'branch', _r1.branch, 'gencost', _r1.gencost);"
)
r1_mpc

In [ ]:
# # NOTE: you also can run like this
# r1_mpc = m.eval(
#     """
#     mpopt = mpoption('verbose', 2);
#     mpc = loadcase('case9');
#     _r1 = runuopf(mpc, mpopt);
#     """
#
#     "struct("
#     " 'baseMVA', _r1.baseMVA, 'version', _r1.version, 'bus', _r1.bus, 'gen', _r1.gen,"
#     " 'branch', _r1.branch, 'gencost', _r1.gencost);"
# )
# r1_mpc